# Text Classification with BERT using Neural Modules - Training

State of the art Native Language Processing (NLP) uses large transformer models like BERT (Bidrectional Encoder Representations from Transformers) to extract meaningful representations from text. These models are pre-trained on a massive corpus of text using unsupervised methods to fill in randomly masked words. The pre-trained BERT model produces embeddings of the text input which then can be used in downstream tasks like text classification, question-answering, and named entity recognition.

In this notebook your task will be text classification, specifically identifying sentences (and the phrases and words they contain) that express a positive or negative opinion. You will load a pre-trained BERT model and enhance it by performing additional training using data from the Stanford Sentiment Treebank. 

You will use NVIDIA Neural Modules (NeMo) to compose the text classification system. NeMo makes state of the art natural language understanding accessible and fast for data scientists. NeMo can automatically download pre-trained BERT models, use single-GPU or multi-GPU training, and leverages powerful optimization techniques like automatic mixed-precision (AMP). 

As part of this workflow, you will load, explore, and process the dataset. Then you will build scalable pipelines for gpu-based training. You will also see how to build inference pipelines that allow you to validate the quality of your models by visualizing the impact of fine-tuning on the learned BERT embeddings.

This workflow also takes advantage the [NetApp AI Control Plane](https://blog.netapp.com/ai-control-plane) to perform data management tasks. In this notebook, you will see how to use the NetApp AI Control Plane to near-instantaneously save and restore versioned dataset and model baselines. You will see how to use this functionality to rollback to previous model versions and to implement dataset to model traceability in a simple manner.

![NLP Fine-tuning](figures/nlp_fine_tuning.png)

Begin by importing all of the required modules/functions/classes.

In [ ]:
import nemo
import nemo.collections.nlp as nemo_nlp
from nemo.collections.nlp.data.datasets import BertTextClassificationDataset
from nemo.collections.nlp.nm.data_layers.text_classification_datalayer import BertTextClassificationDataLayer
from nemo.collections.nlp.nm.trainables import SequenceClassifier

from nemo.backends.pytorch.common import CrossEntropyLossNM
from nemo.utils.lr_policies import get_lr_policy
from nemo.collections.nlp.callbacks.text_classification_callback import eval_iter_callback, eval_epochs_done_callback

import os
import json
import math
import numpy as np
import pandas as pd
pd.options.display.max_colwidth = -1

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
%matplotlib inline

import torch

from netapp_jupyter_utils import netappSnapshotCreate, netappGetSnapshots, netappRestoreSnapshot
from datetime import datetime

You can safely ignore the "UserWarning: Could not import torchaudio. Some features might not work." error this code generates, as this notebook does not utilize any "torchaudio" features.

# NetApp Volume Parameters

Enter the names of the Kubernetes PersistentVolume (PV) objects on which the dataset and model reside. These models are presented as simple folders within the Jupyter workspace through the NetApp AI Control Plane, where the ./data directory is the mounted dataset PV and the ./model directory is the mounted model PV. These PVs were provisioned using NetApp Trident prior to the start of the lab, and are pre-populated with the lab's required dataset and model data.

Notice that the NetApp storage constructs are abstracted away such that the user does not need to know or care that these directories actually represent mounted NetApp volumes, meaning data scientists aren't distracted by storage complexity.

In [ ]:
# Note: PV = PersistentVolume
datasetPvName = 'pvc-3d15d016-854b-4803-9ccc-033be4fa7fa9' # Kubernetes PV on which the dataset resides (mounted at ./data)
modelPvName = 'pvc-b730748b-0510-4bda-9347-380364898f23' # Kubernetes PV on which trained models will be saved (mounted at ./model)

# This just allows Jupyter to re-use these values in the next notebook
%store datasetPvName
%store modelPvName

# Dataset

The [SST-2 dataset](https://nlp.stanford.edu/sentiment/index.html) is a standard benchmark for sentence classification and is part of the [GLUE Benchmark](https://gluebenchmark.com/tasks). As alluded to earlier, the SST-2 dataset has been pre-downloaded from GLUE and extracted into the "data" directory, which is actually a mounted NetApp volume.

Load the dataset.

In [ ]:
WORK_DIR = 'logs'
DATA_DIR = 'data/SST-2'
MODELS_DIR = 'models/'

PRETRAINED_MODEL_NAME = 'bert-base-uncased'
MAX_SEQ_LEN = 64 # we will pad with 0's shorter sentences and truncate longer
BATCH_SIZE = 128

In [ ]:
df = pd.read_csv(DATA_DIR + '/train.tsv', sep='\t')
test_df = pd.read_csv(DATA_DIR + '/test.tsv', sep='\t')

## Data Exploration

Display the first few lines of the dataset, which are sentences extracted from online movie reviews, to get a sense for what the dataset looks like. Each sentence has a label of 0 or 1, indicating whether that sentence is expressing a negative or positive sentiment, respectively.

In [ ]:
df.head()

The dataset comes with a train file (containing labeled values) and a test file (containing unlabeled values).  You will use a portion of the train file contents for training and the rest for model validation. You also need to slightly reformat the test dataset.

In [ ]:
# Split train to train and val and save to disk
np.random.seed(123)
train_mask = np.random.rand((len(df))) < .8
train_df = df[train_mask]
val_df = df[~train_mask]

# Reformat test dataset ("sentence\tlabel")
test_df['label'] = 0
test_df = test_df[['sentence', 'label']]

Save the datasets to disk so that you can later take advantage of NetApp Snapshot technology to save baselined dataset versions. The data transformation you performed here was simple, but these transformations are often complex and may require experimentation with multiple augmentations. NetApp Snapshots facilitate that need by giving you the ability to save off different versions of datasets in a near-instantaneous and highly storage-efficient manner. You will create a snapshot in the next section of this notebook.

In [ ]:
# Save new train, val, and test to disk
SPLIT_DATA_DIR = os.path.join(DATA_DIR, 'split')

os.makedirs(SPLIT_DATA_DIR, exist_ok=True)

train_df.to_csv(os.path.join(SPLIT_DATA_DIR, 'train.tsv'), sep='\t', index=False)
val_df.to_csv(os.path.join(SPLIT_DATA_DIR, 'eval.tsv'), sep='\t', index=False)
test_df.to_csv(os.path.join(SPLIT_DATA_DIR, 'test.tsv'), sep='\t', index=False)

## Save Dataset Baseline for Traceability

Before training the model, use NetApp Snapshot technology to near-instaneously save a baseline version of the dataset for traceability.

First, define a unique tag and a description for the training run. You will use these items to identify the snapshots that correspond to this specific training run. 

In [ ]:
trainingRunTag = 'bert_%s' % datetime.today().strftime('%Y%m%d_%H%M%S') # Training run tag = bert_timestamp
print('trainingRunTag: ', trainingRunTag)
trainingRunDescription = 'BERT training run.'

# This just allows Jupyter to re-use these values in the next notebook
%store trainingRunTag

Now, trigger the creation of a Snapshot copy to save a baseline version of the dataset.

A NetApp Snapshot copy is a read-only, point-in-time image of a volume. The image consumes minimal storage space and incurs negligible performance overhead because it only records file changes that have occurred since the last Snapshot was taken. This makes Snapshot copies ideal for saving versioned baselines of datasets and models, and for implementing dataset-to-model traceability. Since you are using the NetApp AI Control Plane, the underlying NetApp functionality is abstracted away, meaning you can trigger the creation of a Snapshot copy directly from within the notebook rather than having to navigate to an external NetApp-specific tool.

If you want more details on how this NetApp integration works, the "netappSnapshotCreate" method is defined in the netapp_jupyter_utils.py helper script found in the same directory as this notebook. That script leverages ONTAP REST APIs to create the snapshot on the desired PV. If you want to examine that script more closely, you can open the JupyterLab File Browser and double-click that script file to open it in an editor, just be careful not to change the file's contents or you may break the script, which in turn would break the correct operation of this notebook.

In [ ]:
apiResponse, snapshot = netappSnapshotCreate(pvName = datasetPvName, snapshotName = trainingRunTag, snapshotComment = trainingRunDescription)

print('API Response: ', apiResponse['state'])
print('Snapshot uuid: ', snapshot['uuid'])
print('Snapshot name: ', snapshot['name'])
print('Snapshot description: ', snapshot['comment'])
print('Snapshot tag: ', trainingRunTag)

# Neural Modules

In NeMo, everything is a Neural Module. Neural modules abstract data and neural network architectures. A deep learning framework like PyTorch or Tensorflow combines neural network layers to create a neural network, while 
NeMo combines data and neural networks to create AI applications.
The Neural Module Factory manages the neural modules, taking care to flow data through those modules, and it is also responsible for training (including mixed precision and distributed), logging, and inference.

In [ ]:
# instantiate the neural module factory
nf = nemo.core.NeuralModuleFactory(log_dir=WORK_DIR,
                                   create_tb_writer=True,
                                   add_time_to_log_dir=False,
                                   optimization_level='O1')

The pre-trained models are automatically downloaded and cached. Watch your resource utilization tabs as this cell runs. You will see a spike in host network I/O bandwidth utilization, host CPU utilization will increase (but stay relatively low), and GPU memory consumption will increase as the model is loaded into GPU memory.

In [ ]:
# Pre-trained BERT
bert = nemo_nlp.nm.trainables.huggingface.BERT(pretrained_model_name=PRETRAINED_MODEL_NAME)
tokenizer = nemo_nlp.data.NemoBertTokenizer(PRETRAINED_MODEL_NAME)

Note here that the BERT models you are working with are massive. This gives the models a large capacity for learning, which is needed to understand the nuance and complexity of natural language. View the number of model weights.

In [ ]:
print(f'{PRETRAINED_MODEL_NAME} has {bert.num_weights} weights')

In contrast, traditional Multilayer Perceptron (MLP) neural network models are dramatically smaller. Define and instantiate the MLP feed forward network that takes the BERT embeddings as input and outputs the sentence classifications. This model has a much smaller footprint than the BERT model, as indicated by the smaller number of weights.

In [ ]:
# mlp classifier
bert_hidden_size = bert.hidden_size

mlp = SequenceClassifier(hidden_size=bert_hidden_size, 
                         num_classes=2,
                         num_layers=2,
                         log_softmax=False,
                         dropout=0.1)

loss = CrossEntropyLossNM()

In [ ]:
# Compared to the BERT model, the MLP is tiny.
print(f'MLP has {mlp.num_weights} weights')

# Pipelines

Pipelines define how data will flow through the different neural networks. In this case the data will flow through the BERT network and then the MLP network.

Note that this specific use of the term "Pipelines" refers to the general concept of "Pipelines", not to the Kubeflow Pipelines framework.

## Data Layers

Use a pre-built library to convert the data to a format that the BERT model expects.

In [ ]:
train_data = BertTextClassificationDataLayer(input_file=os.path.join(SPLIT_DATA_DIR, 'train.tsv'),
                                             tokenizer=tokenizer,
                                             max_seq_length=MAX_SEQ_LEN,
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             use_cache=True)

val_data = BertTextClassificationDataLayer(input_file=os.path.join(SPLIT_DATA_DIR, 'eval.tsv'),
                                           tokenizer=tokenizer,
                                           max_seq_length=MAX_SEQ_LEN,
                                           batch_size=BATCH_SIZE,
                                           use_cache=True)

In [ ]:
train_input, train_token_types, train_attn_mask, train_labels = train_data()
val_input, val_token_types, val_attn_mask, val_labels = val_data()

# Understanding and Visualizing BERT Embeddings

Create BERT embeddings that convert words into a feature vector, and then use a machine learning algorithm called TSNE to separate a list of example words into categories representing "good" (i.e., positive) words and "bad" (i.e., negative) words.

In [ ]:
# Write example words to file
spectrum_words = ['abysmal', 'apalling', 'dreadful', 'awful', 'terrible',
                  'very bad', 'really bad', 'rubbish', 'unsatisfactory',
                  'bad', 'poor', 'great', 'really good', 'very good', 'awesome'
                  'fantastic', 'superb', 'brilliant', 'incredible', 'excellent'
                  'outstanding', 'perfect']

spectrum_file = os.path.join(SPLIT_DATA_DIR, 'positive_negative.tsv')
with open(spectrum_file, 'w+') as f:
    f.write('sentence\tlabel')
    for word in spectrum_words:
        f.write('\n' + word + '\t0')

spectrum_df = pd.read_csv(spectrum_file, delimiter='\t')

In [ ]:
# Reformat data
spectrum_data = BertTextClassificationDataLayer(input_file=spectrum_file,
                                                tokenizer=tokenizer,
                                                max_seq_length=MAX_SEQ_LEN,
                                                batch_size=BATCH_SIZE)

In [ ]:
# Create word embeddings
spectrum_input, spectrum_token_types, spectrum_attn_mask, spectrum_labels = spectrum_data()
spectrum_embeddings = bert(input_ids=spectrum_input,
                           token_type_ids=spectrum_token_types,
                           attention_mask=spectrum_attn_mask)
spectrum_embeddings_tensors = nf.infer(tensors=[spectrum_embeddings])

Draw a plot to visualize the words that the machine model classifies as expressing a "good" versus "bad" sentiment. 

In [ ]:
spectrum_activations = spectrum_embeddings_tensors[0][0][:,0,:].numpy()
tsne_spectrum = TSNE(n_components=2, perplexity=10, verbose=1, learning_rate=2,
                     random_state=123).fit_transform(spectrum_activations)

fig = plt.figure(figsize=(10,10))
plt.plot(tsne_spectrum[0:11, 0], tsne_spectrum[0:11, 1], 'rx')
plt.plot(tsne_spectrum[11:, 0], tsne_spectrum[11:, 1], 'bo')
for (x,y, label) in zip(tsne_spectrum[0:, 0], tsne_spectrum[0:, 1], spectrum_df.sentence.values.tolist() ):
    plt.annotate(label, # this is the text
                 (x,y), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(0,10), # distance from text to points (x,y)
                 ha='center') # horizontal alignment can be left, right or center

In the preceding plot, "good" words are denoted by a blue dot, and "bad" words are denoted by a red x. If the model is effective, then you should expect "good" words to be closely clustered together and "bad" words to be closely clustered together, with both cluster groups widely separated into the lower-left and upper-right corners. That is not what you see here, so clearly there is room to improve this model, which you will attempt to accomplish shortly through further training.

# Training Pipeline 

## Save Baseline Version of Model

Before training the model, use NetApp Snapshot technology to save a baseline version of the model that you can revert back to in the event that you are not happy with the upcoming training results. Note that the size of a volume and the amount of data it contains does not factor into the time required to create an ONTAP snapshot. 

In [ ]:
modelTag = 'baseline_' + trainingRunTag
modelDescription = 'Baseline pre-trained BERT model.'

bert.config.save_pretrained(MODELS_DIR)
torch.save(bert.state_dict(), MODELS_DIR + 'pytorch_model.bin') # Save pre-trained model to volume

apiResponse, snapshot = netappSnapshotCreate(pvName = modelPvName, snapshotName = modelTag, snapshotComment = modelDescription)

print('API Response: ', apiResponse['state'])
print('Snapshot uuid: ', snapshot['uuid'])
print('Snapshot name: ', snapshot['name'])
print('Snapshot description: ', snapshot['comment'])

## Set up Training Pipeline

Use BERT embeddings to convert from words to a feature vector. This time you are using the full dataset, not just a small sample.

In [ ]:
train_embeddings = bert(input_ids=train_input,
                        token_type_ids=train_token_types,
                        attention_mask=train_attn_mask)
val_embeddings = bert(input_ids=val_input,
                      token_type_ids=val_token_types,
                      attention_mask=val_attn_mask)

Define optimizations and the loss functions.

In [ ]:
NUM_EPOCHS = 3
NUM_GPUS = 1
LEARNING_RATE = 5e-5
OPTIMIZER = 'adam'

In [ ]:
train_logits = mlp(hidden_states=train_embeddings)
val_logits = mlp(hidden_states=val_embeddings)

train_loss = loss(logits=train_logits, labels=train_labels)
val_loss = loss(logits=val_logits, labels=val_labels)

## Callbacks

Callbacks are used to record and log metrics and save checkpoints for the training and evaluation. This notebook also uses callbacks to print results to the display during training execution.

In [ ]:
train_data_size = len(train_data)
steps_per_epoch = math.ceil(train_data_size / (BATCH_SIZE * NUM_GPUS))

train_callback = nemo.core.SimpleLossLoggerCallback(tensors=[train_loss, train_logits],
                            print_func=lambda x:nemo.logging.info(f'Train loss: {str(np.round(x[0].item(), 3))}'),
                            tb_writer=nf.tb_writer,
                            get_tb_values=lambda x: [["train_loss", x[0]]],
                            step_freq=steps_per_epoch)

eval_callback = nemo.core.EvaluatorCallback(eval_tensors=[val_logits, val_labels],
                                            user_iter_callback=lambda x, y: eval_iter_callback(x, y, val_data),
                                            user_epochs_done_callback=lambda x:
                                                eval_epochs_done_callback(x, f'{nf.work_dir}/graphs'),
                                            tb_writer=nf.tb_writer,
                                            eval_step=steps_per_epoch)

# Create callback to save checkpoints
ckpt_callback = nemo.core.CheckpointCallback(folder=nf.checkpoint_dir,
                                             epoch_freq=1)

In [ ]:
lr_policy_fn = get_lr_policy('WarmupAnnealing',
                             total_steps=NUM_EPOCHS * steps_per_epoch,
                             warmup_ratio=0.1)

## Train

Train the neural network. This may take up to 5 minutes. If you set up your notebook dashboard with the resource utilization graphs as described in the accompanying lab guide, then you should see GPU Memory utilization graph max out and hold, and GPU Utilization should ramp up, hold, and then drop multiple times as the training process executes each of the model's three epochs. Host CPU utilization will stat relatively low throughout as the GPU is performing most of the training work.

At the beginning of training and at the end of each of the three training epochs, a callback prints a confusion matrix summarizing that epoch's accuracy results, four matrices in total, with values normalized to the range 0.00 to 1.00. The initial matrix will report accuracy around 0.56, while matrices for later epochs will produce around 0.95, reflecting improved model accuracy.

Once training completes, the cell will output four graphical four-square color confusion matrices summarizing these same results. Purple squares in the lower-left and upper-right quadrants represent a more accurate model. While the first graphic does not match this scheme, later ones do, again indicating that model accuracy has improved during the training.

In [ ]:
%%time
# Run Training
nf.train(tensors_to_optimize=[train_loss],
         callbacks=[train_callback, eval_callback, ckpt_callback],
         lr_policy=lr_policy_fn,
         optimizer=OPTIMIZER,
         optimization_params={'num_epochs': NUM_EPOCHS, 'lr': LEARNING_RATE})

Pause here until the training finishes, as indicated by the contents of the square brackets located to the left of the preceding code cell changing from a '\*' character to a number. The code cell also outputs a series of colored four-square charts when it finishes. 

## Training Conclusion
Congratulations! You have just created a Deep Learning model using the BERT architecture and training. You loaded pre-trained weights from an  online model registry, such as NVIDIA's [NGC](http://ngc.nvidia.com/). You fine-tuned the model for a specific use-case using a real-world dataset, and you performed a basic analysis on the quality of the model.

If you examine your resource utilization tabs, they show that the GPU is idle because the training process has completed. Meanwhile, the GPU Memory is still fully utilized because the model and the weights for the model are still loaded in memory, so that if you needed to perform more training you would not have to re-load this data into GPU RAM again.

Shut down the Jupyter Kernel, which will clear out the system resources, including the GPU memory. 

In [ ]:
import os
os._exit(0)

In the next notebook you will reload this enhanced model and use it to perform an inference.

Please refer back to the lab manual and continue on to the next notebook when you are ready.

### [Click Here for Notebook 2 - Inference](text_classification_with_bert_inference.ipynb)